## Весь препроцессинг вместе

In [174]:
import pandas as pd
import numpy as np
import sklearn as sk
import json as js
import csv

In [175]:
df = pd.read_csv('data/resume.csv', index_col=0)

In [176]:
df.shape

(2369, 19)

In [177]:
df.head(10)

,age,auto,business_trips,categories,citizenship,city,education,employment,experience,experience_length,family,gender,languages,metro_or_moving,name,photo_url,salary,skills,url
0,20,NaN,не готов к командировкам,Архитектура,Россия,Ковров,"[{'kind': 'Неполное высшее образование', 'form...",неполный рабочий день,"[{'period': 'сентябрь 2017 – март 2018', 'leng...",7 месяцев,NaN,f,"['Немецкий язык, Базовый']",NaN,"3D-моделлер, IT стажёр, консультант",NaN,30 000₽,NaN,https://www.superjob.ru/resume/3d-modeller-436...
1,39,['B — легковые авто'],NaN,Техническая поддержка,Россия,Москва,"[{'kind': 'Высшее образование', 'form': 'Дневн...",полная занятость,"[{'period': 'октябрь 2017 – работает сейчас', ...",18 лет,cостоит в браке,m,NaN,NaN,"Администратор, управляющий, IT специалист",https://public.superjob.ru/images/resume_fotos...,50 000₽,[{'professional': '• 1С: Предприятие • 1С: Роз...,https://www.superjob.ru/resume/administrator-2...
2,47,NaN,NaN,Передача данных и доступ в интернет,NaN,Новосибирск,"[{'kind': 'Высшее образование', 'form': 'Вечер...",полная занятость,"[{'period': 'февраль 2008 – работает сейчас', ...",12 лет и 10 месяцев,"cостоит в браке, есть дети",m,"['Английский язык, Технический']",Берёзовая роща,Администратор / Инженер технической поддержки IT,NaN,30 000₽,[{'professional': 'Английский язык уровень. Pr...,https://www.superjob.ru/resume/administrator-2...
3,23,NaN,NaN,Техническая поддержка,NaN,Самара,"[{'kind': 'Высшее образование', 'form': 'Дневн...",сменный график работы,"[{'period': 'июль 2015 – работает сейчас', 'le...",4 года и 3 месяца,NaN,f,NaN,NaN,"Администратор, IT-специалист, бизнес-аналитик,...",https://public.superjob.ru/images/resume_fotos...,25 000₽,"[{'professional': 'Веб-дизайн, photoshop, MS O...",https://www.superjob.ru/resume/administrator-3...
4,52,NaN,не готов к командировкам,АХО,Россия,Ангарск,"[{'kind': 'Высшее образование', 'form': 'Дневн...",NaN,"[{'period': 'июнь 1993 – март 2015', 'length':...",21 год и 10 месяцев,"не состоит в браке, есть дети",f,NaN,NaN,"Администратор, IT-специалист, диспетчер",NaN,20 000₽,NaN,https://www.superjob.ru/resume/administrator-3...
5,22,NaN,NaN,Системное администрирование,Россия,Москва,"[{'kind': 'Неполное высшее образование', 'form...",NaN,"[{'period': 'сентябрь 2017 – декабрь 2017', 'l...",1 год и 2 месяца,NaN,m,NaN,готов к переезду,"Администратор, менеджер, IT сфера",https://public.superjob.ru/images/resume_fotos...,45 000₽,"[{'professional': 'Отличное знание ПК (Exel, 1...",https://www.superjob.ru/resume/administrator-4...
6,23,NaN,NaN,Техническая поддержка,Россия,Миллерово,"[{'kind': 'Среднее специальное образование', '...",полная занятость,"[{'period': 'август 2015 – ноябрь 2017', 'leng...",2 года и 4 месяца,"не состоит в браке, детей нет",m,"['Английский язык, Базовый']",готов к переезду,"Администратор, IT-специалист",https://public.superjob.ru/images/resume_fotos...,20 000₽,[{'professional': 'Поверхностное знание програ...,https://www.superjob.ru/resume/administrator-4...
7,24,"['A — мотоциклы', 'B — легковые авто']",NaN,Сетевые технологии,Россия,Санкт-Петербург,"[{'kind': 'Бакалавр', 'form': 'Дистанционная',...",полная занятость,"[{'period': 'декабрь 2017 – работает сейчас', ...",4 года и 7 месяцев,NaN,m,"['Английский язык, Базовый', 'Эстонский язык, ...",NaN,"Администратор компьютерных сетей, техник IT",NaN,45 000₽,"[{'professional': 'Диагностика, ремонт неиспра...",https://www.superjob.ru/resume/administrator-k...
8,30,NaN,не готов к командировкам,Складское хозяйство,Россия,Железнодорожный,"[{'kind': 'Высшее образование', 'form': 'Очно-...",полная занятость,"[{'period': 'август 2017 – сентябрь 2018', 'le...",6 лет и 2 месяца,cостоит в браке,m,"['Английский язык, Базовый']",NaN,"Администратор магазина, кладовщик, старший кла...",NaN,50 000₽,NaN,https://www.superjob.ru/resume/administrator-m...
9,22,NaN,NaN,NaN,Россия,Рязань,"[{'kind': 'Высшее образование', 'form': 'Очно-...",полная занятость,"[{'period': 'июнь 2014 – август 2018', 'length...",4 года и 3

In [178]:
df.columns

Index(['age', 'auto', 'business_trips', 'categories', 'citizenship', 'city',
       'education', 'employment', 'experience', 'experience_length', 'family',
       'gender', 'languages', 'metro_or_moving', 'name', 'photo_url', 'salary',
       'skills', 'url'],
      dtype='object')

## Обработка колонок:

1. `age` - Возраст
2. `auto` - Наличие прав
3. `business_trips` - Готовность к командировкам $\longrightarrow$ **удалили**
4. `categories` - Категории сайта 
5. `citizenship` - Гражданство 
6. `city` - Город - дало 3 новых признака $\longrightarrow$ `city_msk`, `city_spb`, `city_other`
7. `education` - Образование 
8. `employment` - Желаемая занятость
9. `experience` - Опыт работы
10. `experience_length` - Продолжительность опыта работы
11. `family` - Семейное положение
12. `gender` - Пол
13. `languages` - Владение языками
14. `metro_or_moving` - Станция метро или готовность к переезду
15. `name` - Название вакансии
16. `photo_url` - Ссылка на фотографию  $\longrightarrow$ **удалили**
17. `salary`  - Зарплата
18. `skills` - Ключевые навыки
19. `url` - Ссылка на резюме  $\longrightarrow$ **удалили**

## 1. `age`

In [179]:
 df.age.value_counts(dropna=False)

34    99
35    99
33    98
32    96
31    95
29    93
30    92
41    85
28    85
36    83
27    82
39    75
24    74
37    74
38    73
23    72
26    68
25    67
40    63
22    62
21    61
45    59
43    58
42    56
46    51
44    51
48    45
47    34
49    33
20    31
54    30
52    30
51    27
56    27
55    26
53    26
50    22
58    13
19     9
57     9
59     7
62     5
64     5
60     3
61     3
63     3
18     3
72     2
67     1
66     1
68     1
65     1
70     1
Name: age, dtype: int64

## 2. `auto`

In [180]:
df.auto.value_counts(dropna=False)

['B — легковые авто']                                                                                 1171
NaN                                                                                                    808
['B — легковые авто', 'C — грузовые авто']                                                             220
['A — мотоциклы', 'B — легковые авто']                                                                  74
['A — мотоциклы', 'B — легковые авто', 'C — грузовые авто']                                             52
['C — грузовые авто']                                                                                   11
['B — легковые авто', 'C — грузовые авто', 'D — автобусы']                                              11
['B — легковые авто', 'C — грузовые авто', 'D — автобусы', 'E — авто с прицепом']                        7
['B — легковые авто', 'D — автобусы']                                                                    5
['A — мотоциклы', 'B — легковые авто'

In [181]:
df.auto=df.auto.fillna("['Нет прав']")

## 3. `business_trips`

In [182]:
df.business_trips.value_counts(dropna=False)

NaN                         1373
не готов к командировкам     968
на дому                       28
Name: business_trips, dtype: int64

Много NaN, выкидываем.

In [183]:
df = df.drop('business_trips', axis=1)

## 4. `categories`

In [184]:
df.categories.value_counts(dropna=False)

Системное администрирование                      552
NaN                                              467
Техническая поддержка                            358
IT, Интернет, связь, телеком                     238
Инжиниринг                                       184
Администрирование баз данных                      80
Административная работа, секретариат, АХО         64
Сетевые технологии                                49
Разработка, программирование                      41
Передача данных и доступ в интернет               38
Управление проектами                              34
Аналитика                                         31
Интернет, создание и поддержка сайтов             23
Другое                                            18
Электронный документооборот                       15
Внедрение и сопровождение ПО                      12
Делопроизводство, ввод данных, систематизация     11
Компьютеры, оргтехника, ПО                         9
Call Center                                   

In [185]:
df.categories=df.categories.fillna('Не указано')

## 5. `citizenship`

In [186]:
df.citizenship.value_counts(dropna=False)

Россия               1615
NaN                   727
Украина                 8
Казахстан               7
Кыргызстан              3
Азербайджан             3
Беларусь                2
Таджикистан             1
Литва                   1
Дальнее зарубежье       1
Молдова                 1
Name: citizenship, dtype: int64

## 6. `city`

In [187]:
df.city.value_counts(dropna=False)

Москва                                 720
Санкт-Петербург                        293
Хабаровск                               64
Уфа                                     55
Иркутск                                 51
Ростов-на-Дону                          48
Краснодар                               33
Самара                                  32
Казань                                  27
Саратов                                 27
Рязань                                  26
Нижний Новгород                         24
Пенза                                   23
Ульяновск                               23
Волгоград                               22
Воронеж                                 20
Оренбург                                19
Тверь                                   17
Сыктывкар                               17
Владимир                                16
Смоленск                                16
Екатеринбург                            16
Челябинск                               16
Астрахань  

Оставим 3 признака: Москва, Санкт-Петербург, другие регионы

In [188]:
ctgr = pd.get_dummies(df.city)
ctgr
city_msk = ctgr['Москва']
city_spb = ctgr['Санкт-Петербург']
ctgr = ctgr.drop(['Москва', 'Санкт-Петербург'], axis=1)
city_other = np.sum(ctgr.values, axis=1)
df['city_msk'] = city_Msk
df['city_spb'] = city_Spb
df['city_other'] = city_oth
df = df.drop(['city'], axis=1)
df.shape

(2369, 20)

## 7. `education`

## 8. `employment`

In [189]:
df.employment.value_counts(dropna=False)

полная занятость           1180
NaN                        1107
неполный рабочий день        33
сменный график работы        25
частичная занятость          13
работа вахтовым методом      11
Name: employment, dtype: int64

In [190]:
df.employment=df.employment.fillna("Не указано")

## 9. `experience`

## 10. `experience_length`

In [191]:
df.experience_length.value_counts(dropna=False)

NaN                    126
7 лет и 3 месяца        19
14 лет и 4 месяца       18
5 лет и 10 месяцев      15
10 лет и 6 месяцев      15
11 лет и 4 месяца       15
9 лет и 7 месяцев       15
11 лет и 11 месяцев     15
12 лет и 8 месяцев      15
7 лет и 2 месяца        14
6 лет                   14
13 лет и 11 месяцев     14
10 лет и 2 месяца       14
7 лет и 8 месяцев       14
3 года и 6 месяцев      14
2 года и 5 месяцев      13
16 лет и 2 месяца       13
10 лет и 3 месяца       13
9 лет и 6 месяцев       13
1 год и 4 месяца        13
6 лет и 6 месяцев       13
7 лет и 9 месяцев       12
6 лет и 11 месяцев      12
13 лет и 2 месяца       12
16 лет и 5 месяцев      12
9 лет и 11 месяцев      12
12 лет и 5 месяцев      12
18 лет и 5 месяцев      12
8 лет и 6 месяцев       12
2 года и 10 месяцев     12
                      ... 
19 лет и 10 месяцев      1
25 лет и 4 месяца        1
26 лет и 8 месяцев       1
32 года и 2 месяца       1
38 лет                   1
41 год и 3 месяца        1
2

In [192]:
df = df.dropna(subset=['experience_length'])

In [193]:
df.shape

(2243, 20)

In [194]:
df.index = np.arange(df.shape[0])

In [195]:
def period_to_month(string):
    s = string.split()
    if len(s) == 2:
        if (s[1] == 'год') or (s[1] == 'года') or (s[1] == 'лет'):
            return int(s[0]) * 12
        return int(s[0])
    return int(s[0]) * 12 + int(s[3])

In [196]:
pers = df.experience_length.copy()

In [197]:
pers is df.experience_length

False

In [198]:
for i in pers.index:
    pers.values[i] = period_to_month(pers.values[i])

In [199]:
df.experience_length = pers
df.experience_length

0         7
1       216
2       154
3        51
4       262
5        14
6        28
7        55
8        74
9        51
10       59
11      224
12        3
13       78
14      172
15      361
16      411
17      111
18      250
19      183
20      284
21      216
22      280
23      381
24      186
25      126
26      128
27       42
28      268
29      167
       ... 
2213    121
2214    283
2215    283
2216    207
2217    244
2218    167
2219    121
2220    179
2221    165
2222    389
2223    334
2224    245
2225     97
2226    194
2227    211
2228    154
2229    268
2230     75
2231    298
2232    156
2233    194
2234    339
2235    156
2236    146
2237     66
2238    180
2239    347
2240    229
2241     34
2242     72
Name: experience_length, Length: 2243, dtype: object

## 11. `family`

In [200]:
df.family.value_counts(dropna=False)

cостоит в браке, есть дети       799
NaN                              621
не состоит в браке, детей нет    487
cостоит в браке, детей нет       163
не состоит в браке, есть дети    109
cостоит в браке                   36
не состоит в браке                24
есть дети                          3
детей нет                          1
Name: family, dtype: int64

## 12. `gender`

In [201]:
gender = pd.get_dummies(df.gender)
df['gender'] = gender.f

## 13. `languages`

In [202]:
df.languages.value_counts(dropna=False)

['Английский язык, Технический']                                                                                                                               804
NaN                                                                                                                                                            496
['Английский язык, Базовый']                                                                                                                                   435
['Английский язык, Разговорный']                                                                                                                               157
['Английский язык, Технический', 'Немецкий язык, Базовый']                                                                                                      47
['Английский язык, Базовый', 'Немецкий язык, Базовый']                                                                                                          42
['Английский язык, Cво

## 14. `metro_or_moving`

In [203]:
df.metro_or_moving.value_counts(dropna=False)

NaN                                            1267
готов к переезду                                518
Пражская                                         11
Улица Дыбенко                                     8
Комендантский проспект                            8
Новогиреево                                       6
Ладожская                                         6
Ботанический садВДНХ                              6
Медведково                                        6
Выхино                                            6
Проспект Ветеранов                                6
Планерная                                         5
Коломенская                                       5
Новокосино                                        5
Проспект Большевиков                              5
Академическая                                     5
Щелковская                                        5
Первомайская                                      5
Люблино                                           4
Перово      

## 15. `name`

In [204]:
df.name.value_counts(dropna=False)

Системный администратор, IT-специалист                                                                                                 394
IT-специалист                                                                                                                          178
IT специалист                                                                                                                           67
ИТ-специалист                                                                                                                           48
Руководитель IT-отдела                                                                                                                  37
Инженер IT                                                                                                                              29
Руководитель отдела ИТ                                                                                                                  27
Руководитель IT отдела     

In [205]:
df.name.unique().shape

(1074,)

## 16. `photo_url`

In [206]:
df = df.drop('photo_url', axis=1)

## 17. `salary`

In [207]:
df.salary=df.salary.apply(lambda x: int(x[:-1].replace(" ", '')))

In [208]:
df.salary.value_counts

<bound method IndexOpsMixin.value_counts of 0        30000
1        50000
2        30000
3        25000
4        20000
5        45000
6        20000
7        45000
8        50000
9        30000
10       20000
11       30000
12       35000
13       60000
14       30000
15       75000
16       75000
17       20000
18      150000
19      150000
20       70000
21      280000
22      240000
23       90000
24       80000
25       20000
26       50000
27       50000
28      100000
29      150000
         ...  
2213    175000
2214     80000
2215     80000
2216    150000
2217    200000
2218     50000
2219    120000
2220     70000
2221    130000
2222     50000
2223    120000
2224     55000
2225     50000
2226    100000
2227    150000
2228    120000
2229     50000
2230     25000
2231     60000
2232    100000
2233     50000
2234     50000
2235     60000
2236    110000
2237     30000
2238    150000
2239    150000
2240     40000
2241     45000
2242     80000
Name: salary, Length: 2243, dtype: int64>

## 18. `skills`

In [209]:
df.skills.value_counts(dropna=False).shape

(1882,)

Часть - дубликаты. Но полных дубликатов не было. Видимо, люди хотят на разные должности по одинаковым резюме. Можно оставить как есть (логике задачи это не противоречит)

In [210]:
df.groupby(by='skills')

In [211]:
df[df.skills.isna()].shape

(287, 19)

In [212]:
df = df.dropna(subset=['skills'])

In [213]:
df.shape

(1956, 19)

## 19. `url`

In [214]:
df = df.drop('url', axis = 1)

## Результат:

In [215]:
df.head(10)

,age,auto,categories,citizenship,education,employment,experience,experience_length,family,gender,languages,metro_or_moving,name,salary,skills,city_msk,city_spb,city_other
1,39,['B — легковые авто'],Техническая поддержка,Россия,"[{'kind': 'Высшее образование', 'form': 'Дневн...",полная занятость,"[{'period': 'октябрь 2017 – работает сейчас', ...",216,cостоит в браке,0,NaN,NaN,"Администратор, управляющий, IT специалист",50000,[{'professional': '• 1С: Предприятие • 1С: Роз...,1,0,0
2,47,['Нет прав'],Передача данных и доступ в интернет,NaN,"[{'kind': 'Высшее образование', 'form': 'Вечер...",полная занятость,"[{'period': 'февраль 2008 – работает сейчас', ...",154,"cостоит в браке, есть дети",0,"['Английский язык, Технический']",Берёзовая роща,Администратор / Инженер технической поддержки IT,30000,[{'professional': 'Английский язык уровень. Pr...,0,0,1
3,23,['Нет прав'],Техническая поддержка,NaN,"[{'kind': 'Высшее образование', 'form': 'Дневн...",сменный график работы,"[{'period': 'июль 2015 – работает сейчас', 'le...",51,NaN,1,NaN,NaN,"Администратор, IT-специалист, бизнес-аналитик,...",25000,"[{'professional': 'Веб-дизайн, photoshop, MS O...",0,0,1
5,22,['Нет прав'],Системное администрирование,Россия,"[{'kind': 'Неполное высшее образование', 'form...",Не указано,"[{'period': 'сентябрь 2017 – декабрь 2017', 'l...",14,NaN,0,NaN,готов к переезду,"Администратор, менеджер, IT сфера",45000,"[{'professional': 'Отличное знание ПК (Exel, 1...",1,0,0
6,23,['Нет прав'],Техническая поддержка,Россия,"[{'kind': 'Среднее специальное образование', '...",полная занятость,"[{'period': 'август 2015 – ноябрь 2017', 'leng...",28,"не состоит в браке, детей нет",0,"['Английский язык, Базовый']",готов к переезду,"Администратор, IT-специалист",20000,[{'professional': 'Поверхностное знание програ...,0,0,1
7,24,"['A — мотоциклы', 'B — легковые авто']",Сетевые технологии,Россия,"[{'kind': 'Бакалавр', 'form': 'Дистанционная',...",полная занятость,"[{'period': 'декабрь 2017 – работает сейчас', ...",55,NaN,0,"['Английский язык, Базовый', 'Эстонский язык, ...",NaN,"Администратор компьютерных сетей, техник IT",45000,"[{'professional': 'Диагностика, ремонт неиспра...",0,1,0
9,22,['Нет прав'],Не указано,Россия,"[{'kind': 'Высшее образование', 'form': 'Очно-...",полная занятость,"[{'period': 'июнь 2014 – август 2018', 'length...",51,NaN,0,"['Английский язык, Технический']",NaN,Администратор отдела ГС и ИТ,30000,[{'professional': 'Установка и обслуживание ко...,0,0,1
10,31,"['B — легковые авто', 'C — грузовые авто']",Web-верстка,Россия,"[{'kind': 'Высшее образование', 'form': 'Дневн...",Не указано,"[{'period': 'ноябрь 2018 – работает сейчас', '...",59,не состоит в браке,0,"['Английский язык, Базовый', 'Немецкий язык, Б...",готов к переезду,"Администратор сайта, it специалист",20000,[{'professional': 'Уверенный пользователь. Опы...,0,0,1
11,48,['Нет прав'],Сетевые технологии,Россия,"[{'kind': 'Высшее образование', 'form': 'Дневн...",полная занятость,"[{'period': 'февраль 2014 – апрель 2014', 'len...",224,"cостоит в браке, есть дети",0,"['Английский язык, Технический', 'Русский язык...",NaN,Администратор в области IT,30000,"[{'professional': 'Ответственность, коммуникаб...",0,0,1
12,23,['B — легковые авто'],Техническая поддержка,NaN,"[{'kind': 'Бакалавр', 'form': 'Дневная/Очная',...",полная занятость,"[{'period': 'сентябрь 2018 – работает сейчас',...",3,"не состоит в браке, детей нет",0,"['Английский язык, Базовый']",NaN,"Аналитик, IT специалист",35000,"[{'professional': 'Аналитический склад ума, ув...",1,0,0
